In [3]:
import cymetric as cym
from cymetric import timeseries
import matplotlib as plt 
import pandas as pd
import numpy as np
import sys 
sys.path.insert(0, '../../../scripts/')
import output as oup

# This is when you already have the results 

In [4]:
df = pd.read_csv('ty-df.csv',index_col='TY')
df

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
TY,,,,,,
960,1.295955e+07,7.988186e+08,1.437000e+11,135.083333,962,2
972,1.306918e+07,7.988186e+08,1.437000e+11,120.937493,972,0
984,1.322018e+07,8.033326e+08,1.437000e+11,121.104160,980,0
996,1.336003e+07,8.078466e+08,1.437000e+11,121.104160,980,0
1008,1.333709e+07,8.078466e+08,1.437000e+11,121.104160,980,0


# The rest of the code below is to generate the above results

In [5]:
starter_string = 'TY'
scenario_nums = ['960','972','984','996','1008']

In [ ]:
df = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [ ]:
df['Final HLW'] = 0
df['Final Depleted U'] = 0 
df['Total uranium ore'] = 0
df['Total idle capacity'] = 0
df['Last date idle capacity'] = 0
df['Duration of transition'] = 0

In [ ]:
# Before this is run, you must make sure all the ty_.sqlite files are in the output_start folder. This is done by downloading them from my
# ms-thesis box OR running the transition year cooling time dakota input file. 
output_start = '../cyclus-files/oat/transition-year/ty'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [ ]:
for x in range(5):#(len(scenario_nums)): 
    df.loc[scenario_nums[x],'Final HLW'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['lwrreprocessingwaste','moxreprocessingwaste','frreprocessingwaste'])['Mass'].cumsum().iloc[-1]
    df.loc[scenario_nums[x],'Final Depleted U'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['enrichmentwaste'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total uranium ore'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['sourceout'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total idle capacity'], df.loc[scenario_nums[x],'Last date idle capacity'], udict = oup.idlecap(ev_dict[scenario_nums[x]],'(60000+250*t/12)/1000')
    df.loc[scenario_nums[x],'Duration of transition'] = df.loc[scenario_nums[x],'Last date idle capacity'] - int(scenario_nums[x])
    

In [ ]:
df.to_csv('ty-df.csv')

In [6]:
df_ty_sa = oup.sensitivity(base_case=960,init_df=df)
df_ty_sa

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
TY,,,,,,
960,0.000000,0.000000,0.0,0.000000,0.000000,0.0
972,0.845881,0.000000,0.0,-10.471936,1.039501,-100.0
984,2.011098,0.565086,0.0,-10.348555,1.871102,-100.0
996,3.090223,1.130172,0.0,-10.348555,1.871102,-100.0
1008,2.913157,1.130172,0.0,-10.348555,1.871102,-100.0


In [7]:
df_ty_sa.to_csv('ty-df-sa.csv')